# Jumping frogs puzzle



This project contains my solution to the jumping frogs puzzle. The description of the game together with a demo can be found [here](https://www.primefactorisation.com/frogpuzzle/).


### Description of the solution and proof of correctness

#### Calculation of number of required steps


Let the number of blue and red frogs be $k$ and $p$ respectively. Enumerate the frogs from left to right as $k, k-1, \ldots, 1$ and $-1, -2, \ldots, -p$ (so that all blue frogs have a positive number, while all red frogs have a negative number). Moreover, number the positions on the table by $0, 1, \ldots, p+k$ going from left to right.

Firstly we'll show that we always need exactly $k+p+kp$ steps to solve the puzzle. For this, consider an arbitrary position, and assign to this position the number obtained by summing up the positions of all blue frogs, subtracting the sum of positions of all red frogs, and again subtracting the number of inversions (the number of red-blue frog pairs so that the blue frog is to the left of the red frog).

It can be easily shown that this number increases with exactly $1$ at every step we can take. In the beginning, this value is $\frac{k(k-1)}{2} - kp - \frac{p(p+1)}{2},$ while in the final position it should be $\frac{k(k+1)}{2} - \frac{p(p-1)}{2}.$ Subtracting the two values yields that if the puzzle can be solved, then the number of moves required to reach the final position is exactly $k + p + kp.$

#### The algorithm

The algorithm then goes as follows: in the $m$-th phase, we make a move with either the blue frogs with the $m$ smallest numbers, or with the red frogs with the $m$ smallest numbers (according to the absolute value). The blue and red rounds then alternate after each other.

So, as our first move, we make a move with frog $1.$ Then, in the second phase, we make a move with frogs $-1$ and $-2.$ After that, we move the frogs $1, 2$ and $3.$ Then frogs $-1, -2, -3, -4.$

If $m$ reaches the number of blue or red frogs, it stays at this value until the end of the algorithm.

It can be proven using mathematical induction that following this algorithm, we can always do $k+p+kp$ steps with our frogs. This means that after this many steps we will reach a solution to the puzzle.

#### Running time

The positions of the frogs change dynamically as the program runs. Moreover, we use a dictionaly to store the positions. Therefore we make a constant number of operations each step, so that the running time is $O(kp).$

In [4]:
#algorithm
from termcolor import colored
import numpy as np

class jumping_frog_solver():
    
    def __init__(self, k, p, show_it=True):
        self.k = k
        self.p = p
        self.length = k + p + 1
        self.table = []
        self.positions = {}
        self.initialize()
        self.num_of_moves = 0
        self.moves_to_finish = k + p + k*p
        self.show_it = show_it
        if show_it:
            self.show()

    def solve(self):
        phase = 1
        while True:
            if self.num_of_moves >= self.moves_to_finish:
                print("Number of steps taken: ", self.num_of_moves)
                break
            blues_to_move = min(self.k, phase)
            phase += 1
            reds_to_move = min(self.p, phase)
            phase += 1
            
            self.move_blue_frogs(blues_to_move)
            self.move_red_frogs(reds_to_move)
             
    # the first 'num' many blue frogs should jump
    def move_blue_frogs(self, num):
        for ind in range(num):
            self.move_frog(ind+1)
    
    # the first 'num' many red frogs should jump
    def move_red_frogs(self, num):
        for ind in range(num):
            self.move_frog(-ind-1)
        
    # make the frog jump, if it can
    def move_frog(self, frog):
        i = self.positions[frog]
        
        if self.is_blue(i):
            if self.is_empty(i+1):
                self.move_frog_to(frog, i, i+1)
        
            if self.is_red(i+1) and self.is_empty(i+2):
                self.move_frog_to(frog, i, i+2)
            
        if self.is_red(i):
            if self.is_empty(i-1):
                self.move_frog_to(frog, i, i-1)
        
            if self.is_blue(i-1) and self.is_empty(i-2):
                self.move_frog_to(frog, i, i-2)
    
    # make the frog jump from i to j
    def move_frog_to(self, frog, i, j):
        self.table[i] = 0
        self.table[j] = frog
        self.positions[frog] = j
        self.num_of_moves += 1
        if self.show_it:
            self.show()
        
    def is_blue(self, i):
        if i >= self.length or i < 0:
            return False
        
        return self.table[i] > 0
    
    def is_red(self, i):
        if i >= self.length or i < 0:
            return False
        
        return self.table[i] < 0
    
    def is_empty(self, i):
        if i >= self.length or i < 0:
            return False
        
        return self.table[i] == 0
    
    # to check if we are done
    def we_are_done(self):
        for i in range(self.p):
            if not self.is_red(i):
                return False
            
        for j in range(self.p+1,self.length):
            if not self.is_blue(j):
                return False
            
        return True
       
    def initialize(self):
        for i in range(self.k):
            self.table.append(self.k-i)
            self.positions[self.k-i] = i
            
        self.table.append(0)
        
        for i in range(self.p):
            self.table.append(-i-1)
            self.positions[-i-1] = self.k + 1 + i
        
    def show(self):
        print("{}. step: ".format(self.num_of_moves), end="  ")
        for elem in self.table:
            if elem > 0:
                print(colored(chr(0x2588), "blue"), end="  ")
            elif elem < 0:
                print(colored(chr(0x2588), "red"), end="  ")
            else:
                print(colored(chr(0x2588), "white"), end="  ")
        print("")
        print("-------------------------------------------------------------")

In [5]:
# a concrete example

solver = jumping_frog_solver(4,4)
solver.solve()

0. step:   █  █  █  █  █  █  █  █  █  
-------------------------------------------------------------
1. step:   █  █  █  █  █  █  █  █  █  
-------------------------------------------------------------
2. step:   █  █  █  █  █  █  █  █  █  
-------------------------------------------------------------
3. step:   █  █  █  █  █  █  █  █  █  
-------------------------------------------------------------
4. step:   █  █  █  █  █  █  █  █  █  
-------------------------------------------------------------
5. step:   █  █  █  █  █  █  █  █  █  
-------------------------------------------------------------
6. step:   █  █  █  █  █  █  █  █  █  
-------------------------------------------------------------
7. step:   █  █  █  █  █  █  █  █  █  
-------------------------------------------------------------
8. step:   █  █  █  █  █  █  █  █  █  
-------------------------------------------------------------
9. step:   █  █  █  █  █  █  █  █  █  
----------------------------------------------------

In [6]:
# randomly generated examples

num_of_tests = 20
max_num_of_frogs = 100

for i in range(num_of_tests):
    k = np.random.randint(0,max_num_of_frogs)
    p = np.random.randint(0,max_num_of_frogs)
    solver = jumping_frog_solver(k,p,show_it=False)
    solver.solve()
    assert(solver.we_are_done())

Number of steps taken:  5431
Number of steps taken:  1781
Number of steps taken:  4778
Number of steps taken:  2249
Number of steps taken:  287
Number of steps taken:  1516
Number of steps taken:  5140
Number of steps taken:  1077
Number of steps taken:  1295
Number of steps taken:  6255
Number of steps taken:  4103
Number of steps taken:  944
Number of steps taken:  816
Number of steps taken:  1241
Number of steps taken:  1567
Number of steps taken:  39
Number of steps taken:  939
Number of steps taken:  4887
Number of steps taken:  1678
Number of steps taken:  1357
